<a href="https://colab.research.google.com/github/Ronan-Castro/Pokedex-printable-for-binder/blob/main/Gera_Pokdex_Printer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# === CONFIG ===
start_id = 1
end_id = 151
USE_PORTUGUESE = True  # <-- set to False for English labels


In [27]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!pip install reportlab pillow pandas

In [33]:
import requests
from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.lib.colors import HexColor
from io import BytesIO

# === CONFIGURAÇÕES ===
OUTPUT_PDF = "/content/drive/MyDrive/Pokedex_API.pdf"

# === TYPE COLORS ===
TYPE_COLORS = {
    "grass": "#78C850", "poison": "#A040A0", "fire": "#F08030", "water": "#6890F0",
    "bug": "#A8B820", "normal": "#A8A878", "flying": "#A890F0", "electric": "#F8D030",
    "ground": "#E0C068", "rock": "#B8A038", "psychic": "#F85888", "ice": "#98D8D8",
    "fighting": "#C03028", "ghost": "#705898", "dragon": "#7038F8", "steel": "#B8B8D0",
    "fairy": "#EE99AC", "dark": "#705848"
}

TYPE_TRANSLATIONS = {
    "grass": "Planta", "poison": "Venenoso", "fire": "Fogo", "water": "Água",
    "bug": "Inseto", "normal": "Normal", "flying": "Voador", "electric": "Elétrico",
    "ground": "Terrestre", "rock": "Pedra", "psychic": "Psíquico", "ice": "Gelo",
    "fighting": "Lutador", "ghost": "Fantasma", "dragon": "Dragão", "steel": "Aço",
    "fairy": "Fada", "dark": "Sombrio"
}

# === FETCH DATA FROM POKEAPI ===
def get_pokemon_data(pokemon_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"⚠️ Failed to fetch Pokémon {pokemon_id}")
        return None

    data = response.json()
    name = data["name"].capitalize()
    types_raw = [t["type"]["name"] for t in data["types"]]

    # handle language selection
    if USE_PORTUGUESE:
        types = [TYPE_TRANSLATIONS.get(t, t) for t in types_raw]
    else:
        types = [t.capitalize() for t in types_raw]

    type_text = "/".join(types)
    sprite_url = data["sprites"]["front_default"]
    sprite_image = None

    if sprite_url:
        sprite_image = BytesIO(requests.get(sprite_url).content)

    return {"id": pokemon_id, "name": name, "type": type_text, "types_raw": types_raw, "sprite": sprite_image}

# === DOWNLOAD RANGE ===
pokemon_list = []
for i in range(start_id, end_id + 1):
    p = get_pokemon_data(i)
    if p:
        pokemon_list.append(p)

print(f"{len(pokemon_list)} Pokémon fetched successfully")


# === GENERATE PDF ===
c = canvas.Canvas(OUTPUT_PDF, pagesize=landscape(A4))
width, height = landscape(A4)

cols, rows = 5, 6
margin_x, margin_y = 40, 40
cell_w, cell_h = (width - 2 * margin_x) / cols, (height - 2 * margin_y) / rows

for idx, p in enumerate(pokemon_list):
    page_index = idx % (cols * rows)
    if page_index == 0 and idx != 0:
        c.showPage()

    x = margin_x + (page_index % cols) * cell_w
    y = height - margin_y - ((page_index // cols) + 1) * cell_h

    # === SPRITE ===
    if p["sprite"]:
        try:
            img = ImageReader(p["sprite"])
            iw, ih = img.getSize()
            max_w, max_h = 60, 60
            scale = min(max_w / iw, max_h / ih)
            c.drawImage(img, x + 6, y + cell_h - ih * scale - 8, iw * scale, ih * scale, mask='auto')
        except Exception:
            pass

    # === NAME + NUMBER ===
    c.setFont("Helvetica-Bold", 10)
    text_x = x + 75
    text_y = y + cell_h - 18
    c.setFillColor(HexColor("#000000"))
    c.drawString(text_x, text_y, f"#{p['id']:03d} {p['name']}")

    # === TYPES (aligned boxes) ===
    types = [t.strip() for t in str(p['type']).split('/') if t.strip()]
    spacing, box_h, box_w = 6, 13, 40
    box_y = text_y - 22
    start_x = text_x

    if types:
        # first type box
        t1 = types[0]
        color_hex = TYPE_COLORS.get(p["types_raw"][0], "#DDDDDD")
        c.setFillColor(HexColor(color_hex))
        c.rect(start_x, box_y, box_w, box_h, fill=1, stroke=0)
        c.setFillColor(HexColor("#000000"))
        c.setFont("Helvetica", 8)
        c.drawString(start_x + 4, box_y + 3, t1)

    # second type box (if exists)
    if len(types) > 1:
        t2 = types[1]
        bx2 = start_x + box_w + spacing
        color_hex = TYPE_COLORS.get(p["types_raw"][1], "#DDDDDD")
        c.setFillColor(HexColor(color_hex))
        c.rect(bx2, box_y, box_w, box_h, fill=1, stroke=0)
        c.setFillColor(HexColor("#000000"))
        c.drawString(bx2 + 4, box_y + 3, t2)


c.save()
print(f"PDF generated successfully: {OUTPUT_PDF}")


151 Pokémon fetched successfully
PDF generated successfully: /content/drive/MyDrive/Pokedex_API.pdf
